## Imports

In [3]:
from ipynb.fs.full.Residual import *
from ipynb.fs.full.Jacobian import *

import numpy as np
from numpy import linalg as LA
import sympy as sp

24
0.0200000000000000
f(x) = 6*x
f(x) = 45*x**2 + 36*x


In [4]:
#Time variables
interval = 10

#Number represent how far we want to be precised.
Epsilon = 10**-2

In [5]:
"""
    FirstModel(vec, lg): return an approximation of zero of a 
    function following the Newton's method.
    
    params: - vec : a vector contaning equations (system).
            - lg : vector's size.
    
    return: the approximation calculated.
"""


def FirstModel(vec, lg):
    step = 0
    
    x0 = 1.0
    tab_xk = np.arange(lg, dtype=np.float32)
    tab_fxk = np.arange(lg, dtype=np.float32)
    
    #Loop for the time
    while(step < interval):
        
        #Newton's method
        while(x0 >= Epsilon):
            for i in range(lg):
                #Find the next x for each equations. 
                fxk = evaluate(x0, vec[i])
                fxk_prime = evaluate(x0, Jacobian_one(vec[i]))
                tab_xk[i] = x0 - (float(fxk) / float(fxk_prime))
                
                #Evaluate each x on their respective functions
                tab_fxk[i] = evaluate(tab_xk[i], vec[i])
                
            
            #Calculate the norm.
            #norm = LA.norm(tab_fxk[0])
            """if (tab_fxk[0] - x0 < Epsilon):
                return x0
            else:"""
            x0 -= tab_xk[i]
            

        step+=1
    print("----")
    return tab_xk

In [6]:
vec = [3*x**3-2+6*x + x**2]
res = FirstModel(vec, 1)
print(res)

----
[0.31391707]
